In [ ]:
#Based on https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/2_BasicModels/linear_regression.ipynb
import tensorflow as tf
import numpy
import csv
rng = numpy.random

In [ ]:
# Parameters
learning_rate = 0.01
training_epochs = 1000
display_step = 50

In [ ]:
#Training data
csv = numpy.genfromtxt("solves.csv", skip_header=1, delimiter=",")
train_difficulty = csv[:, 0]
train_features = csv[:, 1:]

#number of training samples
n_samples = train_x.shape[0]

#number of features
feature_length = train_features.shape[1]

In [ ]:

# tf Graph Input
Difficulty = tf.placeholder("float")
Features = tf.placeholder("float", shape=(feature_length))

# Set model weights
W = tf.Variable(tf.zeros([feature_length]), name="weight")
b = tf.Variable(rng.randn(), name="bias")

In [ ]:
# Construct a linear model
pred = tf.add(tf.mul(Features, W), b)

In [ ]:
#TODO: calculate R2 and output it over time
# Mean squared error
with tf.name_scope("cost") as scope:
    cost = tf.reduce_sum(tf.pow(pred-Difficulty, 2))/(2*n_samples)
    cost_summ = tf.scalar_summary("cost", cost)
# Gradient descent
with tf.name_scope("optimize") as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [ ]:
#Summary ops to collect data
#The use of tensorboard is adapted from the example on https://www.tensorflow.org/versions/r0.7/how_tos/summaries_and_tensorboard/index.html
w_hist = tf.histogram_summary("weights", W)
b_hist = tf.histogram_summary("biases", b)
difficulty_hist = tf.histogram_summary("difficulty", Difficulty)

In [ ]:
# Initializing the variables
init = tf.initialize_all_variables()

In [ ]:
# Launch the graph
with tf.Session() as sess:
    
    merged = tf.merge_all_summaries()
    writer = tf.train.SummaryWriter("tmp/linear_logs", sess.graph_def)
    
    sess.run(init)

    # Fit all training data
    for epoch in range(training_epochs):
        for (features, difficulty) in zip(train_features, train_difficulty):
            sess.run(optimizer, feed_dict={Features: features, Difficulty: difficulty})

        #Display logs per epoch step
        if (epoch+1) % display_step == 0:
            result = sess.run([merged,cost], feed_dict={Features: features, Difficulty: difficulty})
            summary_str = result[0]
            c = result[1]
            writer.add_summary(summary_str, epoch)
            print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c), \
                "W=", sess.run(W), "b=", sess.run(b)

    print "Optimization Finished!"
    training_cost = sess.run(cost, feed_dict={Features: features, Difficulty: difficulty})
    print "Training cost=", training_cost, "W=", sess.run(W), "b=", sess.run(b), '\n'


In [ ]:
#TODO: output the model with bias, cost, and weights all zipped up with their names